In [1]:
import string
import boto3

In [2]:
#use boto3 library to feed dataset from s3 to spark

s3 = boto3.resource('s3')
obj=s3.Object('bmtp2','219-0.txt')
file_obj=obj.get()['Body'].read().decode('utf-8')
file_obj=file_obj.replace(u'\ufeff', '') #remove byte order mark from first line
#convert the file string to a list separated by new line
lineSplittedText=file_obj.splitlines()
#convert the text list to spark rdd
text=sc.parallelize(lineSplittedText)

ClientError: An error occurred (ExpiredToken) when calling the GetObject operation: The provided token has expired.

In [ ]:

string.punctuation
text.take(5)

In [3]:
#function to clear punctuation and convert to lower case for proper word count
def clean_text(x):
    lower_case=x.lower() #convert text to lower case
    lower_case=lower_case.replace('--',' ')
    table = str.maketrans(dict.fromkeys(string.punctuation))
    lower_case=lower_case.translate(table)
    return lower_case

In [4]:
#map treats text line by line. For each line, we apply cleaning and split by white space, 
#and flattens the 2d array
tran1= text.flatMap(lambda x: clean_text(x).split())

NameError: name 'text' is not defined

In [5]:
#now map a key and value for each token, key is the word name and value is 1
tran2=tran1.map(lambda x: (x,1))

NameError: name 'tran1' is not defined

In [6]:
tran2.take(4)

NameError: name 'tran2' is not defined

In [9]:
#now reduce by key where the key is word, now you finished word count
tran3=tran2.reduceByKey(lambda x,y: x+y)

In [10]:
tran3.take(3)
result=tran3.collect()
type(result)

list

In [11]:
#Now create a function that appends the list in to a string and writes in to  a file
def storeResult(word_list, filePath):
    data="Word,count \n"
    for pair in word_list:
        data+=pair[1]+"," + str(pair[0]) + "\n"
    _file=open(filePath, 'w')
    _file.write(data)
    _file.close()
    

In [12]:
print(result[0])

('gutenberg', 25)


In [13]:
#now we sort by word to see most frequent words
#first switch keys
tran4=tran3.map(lambda x: (x[1],x[0]))

In [14]:
result=tran4.sortByKey(False).collect()

In [15]:
#Now store the sorted result in a text file, Sorted in descending order from highest to lowest
storeResult(result,"WordCountedS3.csv")